In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [ ]:
%pwd

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [9]:
from src.constants import *
from src.utils.modular import read_yaml_file, create_dir

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml_file(config_filepath)
        self.params=read_yaml_file(params_filepath)

        create_dir([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_dir([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name

        )
        return data_transformation_config

In [12]:
!pip install --upgrade datasets


In [ ]:
import os
from src.logging import logging
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


: 

In [ ]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)